# 3.7 Neural Networks TensorFlow 2

Here we provide a simple example of Neural Network using TensorFlow 2 with MNIST dataset.

Source:

https://www.tensorflow.org/tutorials/quickstart/beginner

In [1]:
import tensorflow as tf

## 1. Load and prepare the MNIST dataset

Reference:

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data

This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. It is considered another classic dataset in machine learning area. The images are handwritings of English letters. Researchers are using it as a training set to build neural network models that would recognize human handwritings.

The dataset we import from tf.keras is not the original image set, but numerical matrix that transfromed from the images via certain kernels.

*tf.keras.datasets.mnist.load_data()* will return tuples of arrays:

- x_train: uint8 NumPy array of grayscale image data with shapes (60000, 28, 28), containing the training data. Pixel values range from 0 to 255.

- y_train: uint8 NumPy array of digit labels (integers in range 0-9) with shape (60000,) for the training data.

- x_test: uint8 NumPy array of grayscale image data with shapes (10000, 28, 28), containing the test data. Pixel values range from 0 to 255.

- y_test: uint8 NumPy array of digit labels (integers in range 0-9) with shape (10000,) for the test data.


In [2]:
# load the dataset
mnist = tf.keras.datasets.mnist

# convert the samples from integers to floating-point numbers
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## 2. Build the `tf.keras.Sequential` model by stacking layers. 

Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.19020575, -0.13285144,  0.3256788 ,  0.04930977, -0.16467923,
        -0.20483004, -0.08942435, -0.83065915, -0.24606049,  0.11837351]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.12810694, 0.09274077, 0.14669247, 0.11127107, 0.08983551,
        0.0863    , 0.09685695, 0.04615477, 0.08281416, 0.11922746]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.4499257

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2994 - accuracy: 0.9119
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1437 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1046 - accuracy: 0.9676
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0860 - accuracy: 0.9735
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0733 - accuracy: 0.9776


## 3. Evaluate the Model

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0743 - accuracy: 0.9774


[0.07432162761688232, 0.977400004863739]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.7540012e-09, 3.5878924e-09, 1.4154824e-05, 3.8316084e-05,
        2.5625644e-11, 1.6074974e-08, 2.4645794e-13, 9.9994731e-01,
        6.8140061e-08, 1.5947148e-07],
       [5.5572391e-06, 6.1343289e-03, 9.9378550e-01, 6.8484755e-05,
        2.2570099e-14, 4.7211329e-06, 1.3188205e-07, 2.8299219e-13,
        1.3076884e-06, 3.2599921e-13],
       [1.3107720e-07, 9.9786550e-01, 1.8182334e-04, 1.7667709e-05,
        2.1234762e-05, 7.9851352e-06, 1.1757026e-05, 6.5722404e-04,
        1.2364595e-03, 2.0259547e-07],
       [9.9978048e-01, 2.8849025e-09, 9.3270064e-05, 7.1077824e-07,
        9.9771678e-06, 6.0668754e-06, 6.2274012e-06, 8.5478481e-05,
        1.3506169e-08, 1.7779306e-05],
       [5.4248569e-05, 3.9520378e-07, 1.9262259e-05, 4.9058854e-06,
        9.8209375e-01, 2.0264627e-06, 6.4437468e-06, 7.7083980e-04,
        1.6315302e-05, 1.7031753e-02]], dtype=float32)>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.